In [20]:
import os

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="RAG-test"

In [21]:
from langchain_openai import ChatOpenAI, OpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=os.environ.get("OPENAI_API_KEY"),temperature=0)

In [22]:
import bs4
from langchain import hub
from langchain_chroma import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

from langchain.prompts import ChatPromptTemplate

In [28]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def vectorstore_from_pdf(pdf_file,chunk_size=1000,chunk_overlap=200,embedding_model = "text-embedding-3-small"):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    splits = text_splitter.split_documents(pages)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model=embedding_model))
    return vectorstore

vectorstore = vectorstore_from_pdf("CCB NS 8415 Underentreprise, avklarende kontraktsm_te, referat.pades.pdf",
                                   chunk_size=250,
                                   chunk_overlap=100)
retriever = vectorstore.as_retriever()

In [29]:
# RAG-Fusion:
template = """ 
Du er en hjelpsom assistent som genererer flere søkeforespørsler basert på en enkelt inngangsforespørsel. \n
Generer flere søkeforespørsler relatert til: {question} \n
Output (3 søkeforespørsler):
"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [30]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

In [37]:
# RAG
from operator import itemgetter
template = """
Du er en assistent som skal sjekke om den 
innkommende etterspørselen fra et selskap med 
krav om betaling er gyldig i henhold til relevante kontraktsdokumenter, 
eller om arbeidet egentlig er inkludert i prising gitt i kontrakten. 
Begrunn hvorfor/hvorfor ikke den innkommende etterspørselen er 
gyldig i henhold til kontraktsdokumentene. 
Dersom kontraktsdokumentet nevner at "prisen er inkludert" betyr dette at selskapet ikke kan kreve mer penger på denne posten.
Begrens svaret til 100 ord.: {context}

Question: {question}
"""

question = """
Tittel
El.kanal gjennom himling

Beskrivelse av forholdet
Det skal monteres el-kanal gjennom himlingene

Utredning og forslag til løsning
Kappet vegglister for el-kanalene i bygg F 5-6-7-8 etg

Vederlagsjustering etter
Regningsarbeid

Kostnadsoversikt
Postnr.	Beskrivelse	Firma	Enhet	Mengde	Enhetspris	Påslag	Totalbeløp	
1	arbeid	OBI	timer	8	660	-	5 280	
Totalsum alle kostnader	5 280
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Den innkommende etterspørselen om å montere el-kanal gjennom himlingene er ikke gyldig i henhold til kontraktsdokumentene. Kontrakten nevner at "Hulltaking i systemvegger for el ektrokanal langs fasade er inkludert", noe som indikerer at arbeidet med el-kanalene allerede er dekket i prisen gitt i kontrakten. Derfor kan selskapet ikke kreve ekstra betaling for dette arbeidet.'